here i attempt to write a REST2 factory for a canonical openmm.system (i.e. containing `HarmonicBondForce`, `HarmonicAngleForce`, `PeriodicTorsionForce`, `NonbondedForce`)

In [1]:
from openmmtools.testsystems import AlanineDipeptideVacuum

define a vacuum alanine dipeptide

In [2]:
ala = AlanineDipeptideVacuum()
sys = ala.system

In [3]:
from perses.annihilation.rest import RESTTopologyFactory

query the topology

In [4]:
ala.topology
for atom in ala.topology.atoms():
    print(atom)

<Atom 0 (H1) of chain 0 residue 0 (ACE)>
<Atom 1 (CH3) of chain 0 residue 0 (ACE)>
<Atom 2 (H2) of chain 0 residue 0 (ACE)>
<Atom 3 (H3) of chain 0 residue 0 (ACE)>
<Atom 4 (C) of chain 0 residue 0 (ACE)>
<Atom 5 (O) of chain 0 residue 0 (ACE)>
<Atom 6 (N) of chain 0 residue 1 (ALA)>
<Atom 7 (H) of chain 0 residue 1 (ALA)>
<Atom 8 (CA) of chain 0 residue 1 (ALA)>
<Atom 9 (HA) of chain 0 residue 1 (ALA)>
<Atom 10 (CB) of chain 0 residue 1 (ALA)>
<Atom 11 (HB1) of chain 0 residue 1 (ALA)>
<Atom 12 (HB2) of chain 0 residue 1 (ALA)>
<Atom 13 (HB3) of chain 0 residue 1 (ALA)>
<Atom 14 (C) of chain 0 residue 1 (ALA)>
<Atom 15 (O) of chain 0 residue 1 (ALA)>
<Atom 16 (N) of chain 0 residue 2 (NME)>
<Atom 17 (H) of chain 0 residue 2 (NME)>
<Atom 18 (C) of chain 0 residue 2 (NME)>
<Atom 19 (H1) of chain 0 residue 2 (NME)>
<Atom 20 (H2) of chain 0 residue 2 (NME)>
<Atom 21 (H3) of chain 0 residue 2 (NME)>


I'll make the ALA residue 'solute' for REST2

remove the `CMMForce`

In [5]:
ala.system.removeForce(4)

make the factory

In [6]:
factory = RESTTopologyFactory(ala.system, solute_region=list(range(6, 16)))

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.


pull the system

In [7]:
REST_system = factory.REST_system

query the forces to make sure it _looks_ right

In [8]:
REST_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ba13fe34cf0> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2ba13fe34ae0> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2ba13fe34060> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2ba174fa5300> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ba174fbdbd0> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ba174fbdc30> >]

all of the nonbonded forces must have the same number of particles

In [9]:
num_particles = REST_system.getNumParticles()
assert num_particles == factory._out_system_forces['CustomNonbondedForce'].getNumParticles()
assert num_particles == factory._out_system_forces['NonbondedForce'].getNumParticles()

## TODO:

here is a few things that have to be done...

- Given the default Global variables, do energy component bookkeeping with the REST system and the original system. Specifically, the first `CustomBondForce` must match the `HarmonicBondForce` (same with the second and third forces with the `HarmonicAngle` and `PeriodicTorsionForce`s). The sum of the `NonbondedForce`, the `CustomNonbondedForce`, and the `CustomBondForce` (last one) must be equal to the energy of the original system's `NonbondedForce`.

- Create a subclass of the `AlchemicalState` to implement the appropriate parameters for `solute_scale` and `inter_scale` (which are `GlobalParameters` of the `REST_system`) and perform Repex.

- Run REST2!

## Run energy component bookkeeping

In [12]:
from openmmtools.states import CompoundThermodynamicState, SamplerState, ThermodynamicState
from simtk import unit
from openmmtools import cache
temp = 300 * unit.kelvin
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
beta = 1/(kB * temp)
from perses.dispersed.feptasks import minimize
from perses.dispersed.utils import configure_platform
from coddiwomple.openmm.utils import get_dummy_integrator
from openmmtools import utils

conducting subsequent work with the following platform: CUDA


In [11]:
from perses.tests.test_topology_proposal import generate_atp

INFO:rdkit:Enabling RDKit 2020.09.1 jupyter extensions


Potential components for the vanilla ala system

In [12]:
thermostate = ThermodynamicState(system=ala.system, temperature=temp)
sampler_state = SamplerState(ala.positions, box_vectors = ala.system.getDefaultPeriodicBoxVectors())
integrator = get_dummy_integrator()
platform = configure_platform(utils.get_fastest_platform().getName())
context = thermostate.create_context(integrator, platform=platform)
minimize(thermostate, sampler_state)
thermostate.reduced_potential(sampler_state)
sampler_state.apply_to_context(context)
print(compute_potential_components(context, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.5874003195280203), ('HarmonicAngleForce', 1.209485556878217), ('PeriodicTorsionForce', 2.321496276111238), ('NonbondedForce', -50.94914523261006), ('AndersenThermostat', 0.0)]


Potential components for the REST system

In [13]:
thermostate = ThermodynamicState(system=REST_system, temperature=temp)
# Don't re-initialize the sampler state 
integrator = get_dummy_integrator()
platform = configure_platform(utils.get_fastest_platform().getName())
context = thermostate.create_context(integrator, platform=platform)
# Don't minimize again
thermostate.reduced_potential(sampler_state)
sampler_state.apply_to_context(context)
print(compute_potential_components(context, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.5874003195280203), ('CustomAngleForce', 1.2094855568782168), ('CustomTorsionForce', 2.3214942161284062), ('NonbondedForce', -1.187543685095446), ('CustomNonbondedForce', -133.23778625838835), ('CustomBondForce', 83.4761858046491), ('AndersenThermostat', 0.0)]


In [ ]:
# REST Nonbonded + CustomNonbonded + CustomBond = -50.949

Potential components for the r-htf hybrid system

In [10]:
import pickle
import os
# Load in r-htf at endstate = 0
out_dir = "/data/chodera/zhangi/perses_benchmark/neq/11/0"
htf = pickle.load(open(os.path.join(out_dir, f"0_vacuum_0.pickle"), "rb" ))

In [13]:
thermostate = ThermodynamicState(system=htf.hybrid_system, temperature=temp)
sampler_state = SamplerState(htf.hybrid_positions, box_vectors = htf.hybrid_system.getDefaultPeriodicBoxVectors())
integrator = get_dummy_integrator()
platform = configure_platform(utils.get_fastest_platform().getName())
context = thermostate.create_context(integrator, platform=platform)
minimize(thermostate, sampler_state)
thermostate.reduced_potential(sampler_state)
sampler_state.apply_to_context(context)
print(compute_potential_components(context, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.7970484134780896), ('HarmonicAngleForce', 2.3577929597628824), ('PeriodicTorsionForce', 24.115351093266632), ('NonbondedForce', -105.77031616495222), ('AndersenThermostat', 0.0)]


Potential components for the REST-ified r-htf hybrid system

In [14]:
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=list(range(6, 16)))

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.


In [17]:
REST_system = factory.REST_system

In [18]:
thermostate = ThermodynamicState(system=REST_system, temperature=temp)
# Don't re-initialize the sampler state 
integrator = get_dummy_integrator()
platform = configure_platform(utils.get_fastest_platform().getName())
context = thermostate.create_context(integrator, platform=platform)
# Don't minimize again
thermostate.reduced_potential(sampler_state)
sampler_state.apply_to_context(context)
print(compute_potential_components(context, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.7970484134780896), ('CustomAngleForce', 2.3577929597628824), ('CustomTorsionForce', 24.11535308037985), ('NonbondedForce', 11.535127952937986), ('CustomNonbondedForce', -104.40538559062273), ('CustomBondForce', -12.900053097452762), ('AndersenThermostat', 0.0)]


In [ ]:
# REST Nonbonded + CustomNonbonded + CustomBond = -105.77

## Run REST2 on capped THR->ALA (flattened torsions/exceptions) in vacuum

In [1]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import copy

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [27]:
# Load in r-htf at endstate = 0
out_dir = "/data/chodera/zhangi/perses_benchmark/neq/11/0"
htf = pickle.load(open(os.path.join(out_dir, f"0_vacuum_0.pickle"), "rb" ))


In [21]:
for i in range(htf.hybrid_system.getNumForces()):
    print(htf.hybrid_system.getForce(i))

<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2afae3592900> >
<simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2afae31d5ba0> >
<simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2afae31d5ba0> >
<simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2afae31d5ba0> >


In [29]:
off_atoms = htf._atom_classes['unique_new_atoms']

In [30]:
off_atoms

{26, 27, 28, 29}

In [31]:
system = htf.hybrid_system
# Check flattened torsions involving off atoms
periodic_torsion = system.getForce(2)
for i in range(periodic_torsion.getNumTorsions()):
    p1, p2, p3, p4, periodicity, phase, k = periodic_torsion.getTorsionParameters(i)
    if p1 in off_atoms or p2 in off_atoms or p3 in off_atoms or p4 in off_atoms:
        print(periodic_torsion.getTorsionParameters(i))
print()
# Check flattened exceptions involving off atoms
nb_force = system.getForce(3)
for i in range(nb_force.getNumExceptions()):
    p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(i)
    if p1 in off_atoms or p2 in off_atoms:
        print(nb_force.getExceptionParameters(i))

[0, 6, 7, 26, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.0, unit=kilojoule/mole)]
[0, 6, 7, 26, 2, Quantity(value=0.0, unit=radian), Quantity(value=0.0, unit=kilojoule/mole)]
[0, 6, 7, 26, 1, Quantity(value=0.0, unit=radian), Quantity(value=0.0, unit=kilojoule/mole)]
[6, 7, 26, 27, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.0, unit=kilojoule/mole)]
[6, 7, 26, 28, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.0, unit=kilojoule/mole)]
[6, 7, 26, 29, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.0, unit=kilojoule/mole)]
[8, 7, 26, 27, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.0, unit=kilojoule/mole)]
[8, 7, 26, 28, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.0, unit=kilojoule/mole)]
[8, 7, 26, 29, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.0, unit=kilojoule/mole)]
[26, 7, 8, 20, 3, Quantity(value=0.0, unit=radian), Quantity(value=0.0, unit=kilojoule/mole)]
[26, 7, 8, 20, 2, Quantity(value=0.0, unit=radian), Quantity(va

In [3]:
# Determine indices for solute region
for atom in htf.hybrid_topology.atoms:
    print(atom.index, atom)

0 ACE1-C
1 ACE1-O
2 ACE1-CH3
3 ACE1-H1
4 ACE1-H2
5 ACE1-H3
6 THR2-N
7 THR2-CA
8 THR2-C
9 THR2-O
10 THR2-CB
11 THR2-CG2
12 THR2-OG1
13 THR2-H
14 THR2-HA
15 THR2-HB
16 THR2-HG1
17 THR2-HG21
18 THR2-HG22
19 THR2-HG23
26 THR2-CB
27 THR2-HB1
28 THR2-HB2
29 THR2-HB3
20 NME3-N
21 NME3-C
22 NME3-H
23 NME3-H1
24 NME3-H2
25 NME3-H3


In [4]:
# Check forces to see if CMMForce needs to be removed
for force in range(htf.hybrid_system.getNumForces()):
    print(htf.hybrid_system.getForce(force))

<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2ab18fcf3de0> >
<simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2ab18fcf3de0> >
<simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2ab1c253bb70> >
<simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2ab18fcf3de0> >


In [3]:
# Build REST factory
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=list(range(6, 10)) + [0, 1, 20, 22])

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.


In [4]:
# Get REST system
REST_system = factory.REST_system

In [5]:
# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 600.0 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

In [6]:
# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTState.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

In [7]:
# Create thermodynamics states
sampler_state =  SamplerState(htf.hybrid_positions, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    print("solute", compound_thermodynamic_state_copy.solute_scale)
    print("inter", compound_thermodynamic_state_copy.inter_scale)
    print()
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

solute 1.0
inter 1.0

solute 0.9468532781436299
inter 0.9730638612874439

solute 0.8947732975651229
inter 0.9459245728730822

solute 0.8439363864122151
inter 0.9186601038535499

solute 0.7945005744165483
inter 0.8913476170476635

solute 0.7466040896140252
inter 0.8640625495958179

solute 0.7003644105543894
inter 0.8368777751585887

solute 0.6558778598246819
inter 0.8098628648263124

solute 0.6132197034187189
inter 0.7830834587824717

solute 0.5724447034546831
inter 0.756600755652995

solute 0.5335880595263061
inter 0.7304711216237819

solute 0.4966666666666667
inter 0.7047458170621992



In [11]:
import numpy as np
T = 600 *unit.kelvin
beta_0 = 1/(kB*T_min)
beta_m = 1/(kB*T)
print("solute", beta_m/beta_0)
print("inter", np.sqrt([beta_m/beta_0]))

solute 0.4966666666666667
inter [0.70474582]


In [14]:
# Set up sampler
length = 5 # 5 ns
move = mcmc.GHMCMove(timestep=4.0*unit.femtoseconds, n_steps=250)
simulation = multistate.ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=length*1000)

# Run t-repex
reporter_file = os.path.join(out_dir, f"0_vacuum_thr_{length}ns.nc")
reporter = multistate.MultiStateReporter(reporter_file, checkpoint_interval=1)
simulation.create(thermodynamic_states=thermodynamic_state_list,
                  sampler_states=sampler_state,
                  storage=reporter)
simulation.run()

DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as /data/chodera/zhangi/perses_benchmark/neq/11/0/0_vacuum_thr_5ns_checkpoint.nc
DEBUG:mpiplus.mpiplus:Cannot find MPI environment. MPI disabled.
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2ab1e9883090>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2ab1c4d660e0>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2ab1c4d17170>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2ab1c4d17440>
DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  5403B | 5.276KB | 0.005MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    0.034s
DEBUG:openmmtools.multistate.multistatesampler:Sto

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

DEBUG:openmmtools.utils:Computing energy matrix took    8.075s
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2ab1e9883090>>
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 1/5000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2ab1c4d56f80>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.029s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 41472/41472 attempted swaps (100.0%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _pr

DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took    0.938s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.048s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2ab1c4d17560>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2ab1c4d17830>
DEBUG:openmmtools.utils:Storing sampler states took    0.002s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.016s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2ab1

DEBUG:openmmtools.multistate.multistatesampler:Iteration 8/5000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2ab1c4d56f80>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.001s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 25414/41472 attempted swaps (61.3%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took    0.954s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.046s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteratio

DEBUG:openmmtools.multistate.multistatesampler:        -0.0    24.2    24.7    24.7    24.7    24.7    24.7    24.7    24.7    24.7    24.7    24.7
DEBUG:openmmtools.utils:Computing online free energy estimate took    0.005s
DEBUG:openmmtools.multistate.multistatesampler:Iteration took 1.028s.
DEBUG:openmmtools.multistate.multistatesampler:Estimated completion in 2:01:43.678044, at Tue Dec 15 16:57:48 2020 (consuming total wall clock time 2:01:59.781563).
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 12/5000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2ab1c4d56f80>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas t

DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2ab1c4d17830>
DEBUG:openmmtools.utils:Storing sampler states took    0.002s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.015s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2ab1c4d1a3b0>
DEBUG:openmmtools.multistate.multistatesampler:*** ONLINE analysis free energies:
DEBUG:openmmtools.multistate.multistatesampler:        -0.0    27.4    27.9    28.0    28.0    28.0    28.0    28.0    28.0    28.0    28.0    28.0
DEBUG:openmmtools.utils:Computing online free energy estimate took    0.005s
DEBUG:openmmtools.multistate.multistatesampler:Iteration took 1.040s.
DEBUG:openmmtools.multistate.multistatesampler:Estimated completion in 1:52:20.396019, at Tue Dec 15 16:48:29 2020 (con

DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took    0.883s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.046s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2ab1c4d17560>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2ab1c4d17830>
DEBUG:openmmtools.utils:Storing sampler states took    0.002s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.015s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2ab1

DEBUG:openmmtools.multistate.multistatesampler:Iteration 23/5000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2ab1c4d56f80>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.001s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 23556/41472 attempted swaps (56.8%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took    0.938s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.046s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iterati

KeyboardInterrupt: 

# Check energies are scaled properly 

In [64]:
from openmmtools.testsystems import AlanineDipeptideVacuum
from perses.annihilation.rest import RESTTopologyFactory
import numpy as np

In [63]:
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
from perses.dispersed import feptasks
from simtk import unit, openmm
from perses.annihilation.lambda_protocol import RESTState
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools.constants import kB

cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())

conducting subsequent work with the following platform: CUDA


RuntimeError: Cannot change platform of a non-empty ContextCache

At 600 K, check that energy of REST system matches that of a vanilla system where I manually scale the terms by the same factor

In [83]:
ala = AlanineDipeptideVacuum()
sys = ala.system
sys.removeForce(4)

In [84]:
factory = RESTTopologyFactory(sys, solute_region=list(range(6, 16)))

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.


In [85]:
REST_system = factory.REST_system

In [86]:

# Create states for 600K
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 600.0 * unit.kelvin  # Maximum temperature.

# Thermodynamic state
lambda_zero_alchemical_state = RESTState.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_max)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

beta_0 = 1/(kB*T_min)
beta_m = 1/(kB*T_max)
compound_thermodynamic_state.set_alchemical_parameters(beta_0, beta_m)

# Now generating a sampler_state for the thermodyanmic state, with relaxed positions
sampler_state = SamplerState(ala.positions, box_vectors=sys.getDefaultPeriodicBoxVectors())
context_cache = cache.ContextCache()
context, context_integrator = context_cache.get_context(compound_thermodynamic_state)
feptasks.minimize(compound_thermodynamic_state, sampler_state)
compound_thermodynamic_state.reduced_potential(sampler_state)


-12.041480001960524

In [87]:
for i in range(sys.getNumForces()):
    print(sys.getForce(i))

<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2abbb9c7b6c0> >
<simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2abbb9c7b450> >
<simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2abbb9c7b6c0> >
<simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2abbb9c7b450> >


In [88]:
# Determine regions and scaling factors
protein = range(6, 16)
environment = list(range(0, 6)) + list(range(16, 22))
protein_scale = beta_m/beta_0
inter_scale = np.sqrt(beta_m/beta_0)


In [89]:
# Scale the terms in the vanilla ala system bond force appropriately
bond_force = sys.getForce(0)
for bond in range(bond_force.getNumBonds()):
    p1, p2, length, k = bond_force.getBondParameters(bond)
    if p1 in protein and p2 in protein:
        bond_force.setBondParameters(bond, p1, p2, length, k*protein_scale)
    elif (p1 in protein and p2 in environment) or (p1 in environment and p2 in protein):
        bond_force.setBondParameters(bond, p1, p2, length, k*inter_scale)
#     elif p1 in environment and p2 in environment:
        

In [90]:
# Scale the terms in the vanilla ala system angle force appropriately
angle_force = sys.getForce(1)
for angle_index in range(angle_force.getNumAngles()):
    p1, p2, p3, angle, k = angle_force.getAngleParameters(angle_index)
    if p1 in protein and p2 in protein and p3 in protein:
        angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k*protein_scale)
#         print("protein scale: ", p1, p2, p3)
    elif set([p1, p2, p3]).intersection(set(protein)) != set() and set([p1, p2, p3]).intersection(set(environment)) != set():
        angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k*inter_scale)
#         print("inter scale: ", p1, p2, p3)
        

In [91]:
# Scale the terms in the vanilla ala system torsion force appropriately
torsion_force = sys.getForce(2)
for torsion_index in range(torsion_force.getNumTorsions()):
    p1, p2, p3, p4, periodicity, phase, k = torsion_force.getTorsionParameters(torsion_index)
    if p1 in protein and p2 in protein and p3 in protein and p4 in protein:
        torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k*protein_scale)
#         print("protein scale: ", p1, p2, p3, p4)
    elif set([p1, p2, p3, p4]).intersection(set(protein)) != set() and set([p1, p2, p3, p4]).intersection(set(environment)) != set():
        torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k*inter_scale)
#         print("inter scale: ", p1, p2, p3, p4)
        

In [92]:
# Scale the terms in the vanilla ala system nonbonded force appropriately
nb_force = sys.getForce(3)
for nb_index in range(nb_force.getNumParticles()):
    charge, sigma, epsilon = nb_force.getParticleParameters(nb_index)
    if nb_index in protein:
        nb_force.setParticleParameters(nb_index, protein_scale*charge, sigma, protein_scale*epsilon)


In [93]:
# Scale the exceptions in the vanilla ala system nonbonded force appropriately
nb_force = sys.getForce(3)
for nb_index in range(nb_force.getNumExceptions()):
    p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(nb_index)
    if p1 in protein and p2 in protein:
        nb_force.setExceptionParameters(nb_index, p1, p2, protein_scale*chargeProd, sigma, protein_scale*epsilon)
    elif (p1 in protein and p2 in environment) or (p1 in environment and p2 in protein):
        nb_force.setExceptionParameters(nb_index, p1, p2, inter_scale*chargeProd, sigma, inter_scale*epsilon)
   

In [94]:
# Check energy
thermostate = ThermodynamicState(sys, temperature=T_max)
context_cache = cache.ContextCache()
context, context_integrator = context_cache.get_context(thermostate)
thermostate.reduced_potential(sampler_state)

-12.041480001960524

At 600 K, check that energy of REST-ified rhtf hybrid system matches that of the rhtf hybrid system where I manually scale the terms by the same factor

In [67]:
import pickle
import os

In [68]:
# Load in r-htf at endstate = 0
out_dir = "/data/chodera/zhangi/perses_benchmark/neq/11/0"
htf = pickle.load(open(os.path.join(out_dir, f"0_vacuum_0.pickle"), "rb" ))
sys = htf.hybrid_system

In [69]:
factory = RESTTopologyFactory(sys, solute_region=list(range(0, 16)))

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.


In [70]:
REST_system = factory.REST_system

In [72]:
# Create states for 600K
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 600.0 * unit.kelvin  # Maximum temperature.

# Thermodynamic state
lambda_zero_alchemical_state = RESTState.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_max)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

beta_0 = 1/(kB*T_min)
beta_m = 1/(kB*T_max)
compound_thermodynamic_state.set_alchemical_parameters(beta_0, beta_m)

# Now generating a sampler_state for the thermodyanmic state, with relaxed positions
sampler_state = SamplerState(htf.hybrid_positions, box_vectors=sys.getDefaultPeriodicBoxVectors())
context_cache = cache.ContextCache()
context, context_integrator = context_cache.get_context(compound_thermodynamic_state)
feptasks.minimize(compound_thermodynamic_state, sampler_state)
compound_thermodynamic_state.reduced_potential(sampler_state)


-16.933324794808108

In [19]:
for i in range(sys.getNumForces()):
    print(sys.getForce(i))

<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2abb708fa330> >
<simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2abb2b67c030> >
<simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2abb2b67c030> >
<simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2abb2b67c030> >


In [73]:
# Determine regions and scaling factors
protein = range(6, 16)
environment = list(range(0, 6)) + list(range(16, 22))
protein_scale = beta_m/beta_0
inter_scale = np.sqrt(beta_m/beta_0)


In [74]:
# Scale the terms in the vanilla ala system bond force appropriately
bond_force = sys.getForce(0)
for bond in range(bond_force.getNumBonds()):
    p1, p2, length, k = bond_force.getBondParameters(bond)
    if p1 in protein and p2 in protein:
        bond_force.setBondParameters(bond, p1, p2, length, k*protein_scale)
    elif (p1 in protein and p2 in environment) or (p1 in environment and p2 in protein):
        bond_force.setBondParameters(bond, p1, p2, length, k*inter_scale)
#     elif p1 in environment and p2 in environment:
        

In [75]:
# Scale the terms in the vanilla ala system angle force appropriately
angle_force = sys.getForce(1)
for angle_index in range(angle_force.getNumAngles()):
    p1, p2, p3, angle, k = angle_force.getAngleParameters(angle_index)
    if p1 in protein and p2 in protein and p3 in protein:
        angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k*protein_scale)
#         print("protein scale: ", p1, p2, p3)
    elif set([p1, p2, p3]).intersection(set(protein)) != set() and set([p1, p2, p3]).intersection(set(environment)) != set():
        angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k*inter_scale)
#         print("inter scale: ", p1, p2, p3)
        

In [76]:
# Scale the terms in the vanilla ala system torsion force appropriately
torsion_force = sys.getForce(2)
for torsion_index in range(torsion_force.getNumTorsions()):
    p1, p2, p3, p4, periodicity, phase, k = torsion_force.getTorsionParameters(torsion_index)
    if p1 in protein and p2 in protein and p3 in protein and p4 in protein:
        torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k*protein_scale)
#         print("protein scale: ", p1, p2, p3, p4)
    elif set([p1, p2, p3, p4]).intersection(set(protein)) != set() and set([p1, p2, p3, p4]).intersection(set(environment)) != set():
        torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k*inter_scale)
#         print("inter scale: ", p1, p2, p3, p4)
        

In [77]:
# Scale the terms in the vanilla ala system nonbonded force appropriately
nb_force = sys.getForce(3)
for nb_index in range(nb_force.getNumParticles()):
    charge, sigma, epsilon = nb_force.getParticleParameters(nb_index)
    if nb_index in protein:
        nb_force.setParticleParameters(nb_index, protein_scale*charge, sigma, protein_scale*epsilon)


In [78]:
# Scale the exceptions in the vanilla ala system nonbonded force appropriately
nb_force = sys.getForce(3)
for nb_index in range(nb_force.getNumExceptions()):
    p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(nb_index)
    if p1 in protein and p2 in protein:
        nb_force.setExceptionParameters(nb_index, p1, p2, protein_scale*chargeProd, sigma, protein_scale*epsilon)
    elif (p1 in protein and p2 in environment) or (p1 in environment and p2 in protein):
        nb_force.setExceptionParameters(nb_index, p1, p2, inter_scale*chargeProd, sigma, inter_scale*epsilon)
   

In [79]:
# Check energy
thermostate = ThermodynamicState(sys, temperature=T_max)
context_cache = cache.ContextCache()
context, context_integrator = context_cache.get_context(thermostate)
thermostate.reduced_potential(sampler_state)

-16.933324794808108